# essiHack Team 

### This notebook demonstrates how to use the `semanticanalyser-py` and `dab-py` packages to perform semantic analysis of terms from BDIs within a specified geographical area.

- **`semanticanalyser-py`**: This package provides a Python interface for interacting with the Semantic Analyser service maintained by BODC. It allows you to analyse terms and find related concepts.
- **`dab-py`**: This package provides tools for interacting with the Geo Data Abstraction Broker (DAB) service. In this notebook, it is used to call the terms API to retrieve terms based on a bounding box.

The notebook utilizes `ipyleaflet` to display an interactive map where you can draw a rectangle to define the area of interest. The terms within this area for a selected category (parameter, instrument, or platform) are then retrieved and displayed. You can select these terms and perform semantic analysis on them using the `semanticanalyser-py` package.

### Libraries needed for our project

In [23]:
# Libraries available on  our Github: 
# https://github.com/ESSI-Lab/semanticanalyser-py
# https://github.com/ESSI-Lab/dab-py
!pip install --upgrade semanticanalyser-py
!pip show semanticanalyser-py
!pip install --upgrade dab-py

# Map libraries
!pip install folium ipyleaflet

Name: semanticanalyser-py
Version: 0.1.7
Summary: A Python binding for the Semantic Analyser service maintained by BODC
Home-page: https://github.com/ESSI-Lab/semanticanalyser-py
Author: Ahmad Mahmoud (CNR internship)
Author-email: ahmad.mahmoud@edu.unifi.it
License: GPL-3.0
Location: /opt/conda/lib/python3.12/site-packages
Requires: requests
Required-by: 


### The main code

In [24]:
from IPython.display import display, HTML
from semanticanalyser.analyser import *
from ipyleaflet import Map, Rectangle, DrawControl
from dabpy import *
from ipywidgets import SelectMultiple, VBox, Output, Button, GridspecLayout, Label, Layout, Checkbox, Dropdown
import requests
import json


# Change the map center and zoom level to focus on Europe
m = Map(center=(50, 12), zoom=3)
# Add a DrawControl to the map to allow drawing shapes, specifically a rectangle
draw_control = DrawControl(rectangle={"shapeOptions": {"color": "#0000FF"}})
m.add_control(draw_control)


# Initialize the category dropdown for selecting term types (parameter, instrument, platform)
category_dropdown = Dropdown(
    options=['parameter', 'instrument', 'platform'],
    value='instrument',
    description='Category:',
    disabled=False,
    layout=Layout(width='95%')
)

# Initialize the search and analyse buttons
search_button = Button(description="Search")
# Initialize the SelectMultiple widget to display the retrieved terms
terms_select_multiple = SelectMultiple(
    options=[],
    description='Select Terms:',
    disabled=False,
    layout=Layout(width='95%')
)

# Initialize the analyse button
analyse_button = Button(description="Analyse")

# Variable to store the coordinates of the drawn rectangle
rectangle_coordinates = None


# Dictionary to store the full term data with 'value' as the key for easy lookup
full_terms_data = {}

# Initialize output widgets for displaying analysis results and logs
output_widget = Output()
log_output_widget = Output()

# Create a grid layout to arrange the widgets
grid = GridspecLayout(4, 2, height='600px')
# Place the category dropdown, search button, terms select multiple, and analyse button in the first column
grid[0:2, 0] = VBox([category_dropdown, search_button, terms_select_multiple, analyse_button], layout=Layout(width='100%'))
# Place the log output widget in the second column
grid[0:2, 1] = log_output_widget
# Place the main output widget for analysis results across both columns below the other widgets
grid[2:, 0:] = output_widget

# Display the map and the grid layout
display(m)
display(grid)


def call_terms_api(bbox_coordinates):
    """
    Calls the terms API with the given bounding box coordinates and displays terms in a SelectMultiple widget.

    Args:
        bbox_coordinates (list): A list containing the coordinates of the bounding box.
    """
    global full_terms_data
    if not bbox_coordinates:
        with log_output_widget:
            log_output_widget.clear_output()
            print("No rectangle drawn yet.")
        terms_select_multiple.options = []
        full_terms_data = {}
        return

    # Extract latitude and longitude from the bounding box coordinates
    lat1 = bbox_coordinates[0][1]
    lon1 = bbox_coordinates[0][0]
    lat2 = bbox_coordinates[1][1]
    lon2 = bbox_coordinates[1][0]

    # Determine the north, south, east, and west boundaries
    north = max(lat1, lat2)
    south = min(lat1, lat2)
    east = max(lon1, lon2)
    west = min(lon1, lon2)

    # Define the token and view for the API call
    token = "blue-cloud-terms-maris-bv"
    view = "blue-cloud-terms"
    # Get the selected category from the dropdown
    selected_category = category_dropdown.value

    # Construct the API URL
    api_url = f"https://gs-service-preproduction.geodab.eu/gs-service/services/essi/token/{token}/view/{view}/terms-api/terms"

    # Define the parameters for the API request
    params = {
        "type": selected_category,
        "west": west,
        "south": south,
        "east": east,
        "north": north,
        "offset": 1,
        "limit": 200 # Limit the number of results
    }

    # Make the API call and handle the response
    with log_output_widget:
        log_output_widget.clear_output()
        print("Calling terms API with URL:", api_url)
        print("Parameters:", params)
        try:
            response = requests.get(api_url, params=params)
            response.raise_for_status() # Raise an exception for bad status codes
            data = response.json()
            # Extract the terms data from the response
            terms_data = data.get("terms", []) if isinstance(data, dict) else data

            # Process the terms data if it's a list
            if isinstance(terms_data, list):
                # Store the full term data in a dictionary for easy lookup by value
                full_terms_data = {str(item.get('value', '')): item for item in terms_data if isinstance(item, dict)}
                # Get the list of term values to display in the SelectMultiple widget
                terms_list = list(full_terms_data.keys())
                terms_select_multiple.options = terms_list
                print(f"Received {len(terms_list)} terms for category: {selected_category}.")
            else:
                print(f"API response does not contain a list of terms for category: {selected_category}.")
                terms_select_multiple.options = []
                full_terms_data = {} # Clear data if response is not a list

        except requests.exceptions.RequestException as e:
            # Handle request-specific errors
            print(f"Error calling API: {e}")
            terms_select_multiple.options = []
            full_terms_data = {}
        except Exception as e:
             # Handle any other unexpected errors
             print(f"An unexpected error occurred: {e}")
             terms_select_multiple.options = []
             full_terms_data = {}


def on_search_button_clicked(b):
    """Handler for the search button click event. Calls the terms API with the current rectangle coordinates."""
    with log_output_widget:
      log_output_widget.clear_output()
    call_terms_api(rectangle_coordinates)

# Attach the handler function to the search button's click event
search_button.on_click(on_search_button_clicked)


def on_analyse_button_clicked(b):
    """
    Handler for the analyse button click event.
    Performs semantic analysis on the selected terms.
    """
    # Get the selected values from the SelectMultiple widget
    selected_values = terms_select_multiple.value

    # Check if any terms are selected
    if not selected_values:
        with output_widget:
            output_widget.clear_output()
            print("No terms selected for analysis.")
        return

    with log_output_widget:
        log_output_widget.clear_output()
        # Convert selected values to a list
        selected_terms_list = list(selected_values)

        print("Selected values for analysis:", selected_terms_list)

        try:
            print("Terms being sent for analysis:", selected_terms_list)

            # Initialize the SemanticAnalyser
            analyser = SemanticAnalyser()
            # Get available match types and properties from the analyser service
            match_types = analyser.getMatchTypes()
            match_properties = analyser.getMatchProperties()
            # Get the selected category
            selected_category = category_dropdown.value
            print()
            print("Analyse terms called with the following parameters:")
            print("Terms: ", selected_terms_list)
            print("Match Types: ")
            for mt in match_types:
                print(f"  {mt}")
            print("Match properties: ")
            for mp in match_properties:
                print(f"  {mp}")
            print("Category:", selected_category) # Print selected category


            with output_widget:
                output_widget.clear_output()
                # Perform the semantic analysis
                analysis_result = analyser.analyseTerms(selected_terms_list, match_types, match_properties, selected_category)
            # Process and display the analysis result
            if analysis_result is not None:
                with output_widget:
                    try:
                        # Convert the analysis result to a string
                        result_string = analysis_result.to_string()
                        if result_string:
                            # Define CSS style for the output
                            css_style = """
                            <style>
                                .analysis-output {
                                    font-family: sans-serif;
                                    padding: 10px;
                                    border: 1px solid #ccc;
                                    border-radius: 5px;
                                    margin-top: 10px;
                                }
                                .analysis-output h3 {
                                    color: #0056b3;
                                    margin-top: 0;
                                    margin-bottom: 5px;
                                }
                                .analysis-output ul {
                                    list-style-type: none;
                                    padding: 0;
                                    margin: 0;
                                }
                                .analysis-output li {
                                    margin-bottom: 5px;
                                }
                                .analysis-output a {
                                    color: #007bff;
                                    text-decoration: none;
                                }
                                .analysis-output a:hover {
                                    text-decoration: underline;
                                }
                                .analysis-output hr {
                                    border: 0;
                                    height: 1px;
                                    background: #ccc;
                                    margin: 15px 0;
                                }
                            </style>
                            """
                            # Wrap the result string in a div with the class for styling
                            styled_html = f"{css_style}<div class='analysis-output'>{result_string}</div>"
                            display(HTML(styled_html)) # Display HTML directly
                        else:
                            print("Semantic analysis did not return any matches.")
                    except json.JSONDecodeError:
                        print("Could not decode JSON from analysis API response.")
                        print("Raw analysis API response:", analysis_result.text)
            else:
                with output_widget:
                    print("Semantic analysis did not return a result.")


        except requests.exceptions.RequestException as e:
            with output_widget:
                print(f"Error during analysis request: {e}")
        except Exception as e:
            with output_widget:
                print(f"Error during semantic analysis processing: {e}")

# Attach the handler function to the analyse button's click event
analyse_button.on_click(on_analyse_button_clicked)



# Callback function to handle drawing events on the map
def handle_draw(self, action, geo_json):
    """
    Handles drawing events on the map. Captures the coordinates of a created rectangle.

    Args:
        action (str): The type of drawing action (e.g., 'created').
        geo_json (dict): The GeoJSON object representing the drawn shape.
    """
    global rectangle_coordinates
    # Check if the action is 'created' and the shape is a Polygon (rectangle)
    if action == 'created' and geo_json['geometry']['type'] == 'Polygon':
        # Get the coordinates of the polygon's points
        points = geo_json['geometry']['coordinates'][0]
        # If there are at least two points, assume a rectangle and store the first and third points as corners
        if len(points) >= 2:
             rectangle_coordinates = [points[0], points[2]]
        else:
             rectangle_coordinates = None

        # Log the captured coordinates
        with log_output_widget:
            log_output_widget.clear_output()
            if rectangle_coordinates:
                print("Rectangle coordinates captured:", rectangle_coordinates)
            else:
                 print("Could not capture rectangle coordinates.")

# Attach the handle_draw function to the draw control's draw event
draw_control.on_draw(handle_draw)

Map(center=[50, 12], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

GridspecLayout(children=(VBox(children=(Dropdown(description='Category:', index=1, layout=Layout(width='95%'),…

### How to use the map 

This section of the notebook provides a user interface for interacting with the Semantic Analyser and DAB services.

You will see a map where you can draw a rectangle to define your area of interest. Below the map, there is a panel with the following controls:

- **Category dropdown list:** Use this to select the type of terms you want to search for (parameter, instrument, or platform).
- **Search button:**  after drawing a rectangle on the map and selecting a category to retrieve relevant terms within that area, Click this button to search for terms using the DAB API.
- **Select Terms box:** The terms found will appear in this box. You can select a term from the list.
- **Analyse button:** After selecting terms, click this button to perform a semantic analysis on the chosen term.
- **Output area:** The results of the semantic analysis will be displayed here.
- **Log Output area (top right):** This area shows messages about the process, such as API calls being made.

To use the interface:
1. Draw a rectangle on the map to define your search area.
2. Select a category from the dropdown.
3. Click the "Search" button to load terms.
4. Select the terms you want to analyse from the list.
5. Click the "Analyse" button to see the results.